In [1]:
import requests
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from urllib.parse import urlencode
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select

# Scraping jobs numbers

In [12]:
# download excel file from the link in url
url="https://dol.ny.gov/statistics-cessaxls"
r= requests.get(url, allow_redirects=True)
open('data/raw_employment_data.xlsx', 'wb').write(r.content)
# read the file
df=pd.read_excel('data/raw_employment_data.xlsx', sheet_name='Metro Areas')
# file out new York City data
df=df[df.iloc[:,1] =="New York City"].reset_index(drop=True)
# create column names here
column_names=['job_type','area','year', "January","February","March","April","May",
"June","July","August","September","October","November","December"]
# insert new columns names into the original dataset
df.columns=column_names
# filter out columns that aren't needed
df=df[['year', "January","February","March","April","May","June","July","August","September",
    "October","November","December"]]
# flatten the data to. make it graphics ready
df=df.melt(id_vars=['year'])
# create a datetime column for viz purposes
df['date']=pd.to_datetime(df.year.astype(str)+"-"+df.variable)
# replace  commas from the value column so that we can convert it into a string later
df=df.replace(",", "", regex=True)
# rename value column
df=df[['date','value' ]].rename(columns={'value':'jobs'})
# flter to get all entries after Jan 2020
df=df[df.date > "2020-01-31"].reset_index(drop=True)
# remove empty columns
df['jobs']=df.jobs.replace(" ", np.nan).astype(float)
# drop na values
df=df[df.jobs.notna()].reset_index(drop=True)
# multiple by 1000 to create original value
df['jobs']=(df.jobs*1000).astype(int)
# sort columbs by datetime
df=df.sort_values('date').reset_index(drop=True)
#create a job loss column from baseline: Feb 2020
df['jobloss_from_feb2020']=(df['jobs']-4715100.0).astype(int)
#save file in the data folder
df['date']=df.date.astype(str)
df.to_json('data/job_recovery.json', orient='records')
df.to_csv('data/job_recovery.csv', index=False)

# Jobs by sectors

In [74]:
# s=Service('/Applications/chromedriver')
# driver=webdriver.Chrome(service=s)

# industry_url='https://statistics.labor.ny.gov/cesemp.asp'
# driver.get(industry_url)

# try:
#     dropdown = driver.find_element_by_id("codename")
#     print('Successfully logged in')
# except NoSuchElementException:
#     print('Incorrect dropdown')
    
# nyc=Select(dropdown)
# nyc_select=nyc.select_by_value("21093561")
# nyc_page=driver.find_element(
#     by=By.XPATH, value=
#     '//*[@id="ux-page"]/div/div[3]/div[1]/div/form/div/fieldset/div/input').click()

/var/folders/09/0pcf7rgs7c38nk2clfp7p_5w0000gq/T/ipykernel_20838/3013117398.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  dropdown = driver.find_element_by_id("codename")


Successfully logged in


In [11]:
# industry_list=[]
# total_elems=len(driver.find_elements_by_tag_name('tr'))
# for tr in range(1,(total_elems+1)):
#     industry_dict={}
#     industry_dict['industry']=BeautifulSoup(driver.find_element(
#             by=By.XPATH, value=f'//*[@id="ux-page"]/div/div[2]/strong/table/tbody/tr[{tr}]/td[1]').text).text
#     industry_dict['current_value']=BeautifulSoup(driver.find_element(
#             by=By.XPATH, value=f'//*[@id="ux-page"]/div/div[2]/strong/table/tbody/tr[{tr}]/td[2]').text).text
#     industry_dict['comp_value']=BeautifulSoup(driver.find_element(
#             by=By.XPATH, value=f'//*[@id="ux-page"]/div/div[2]/strong/table/tbody/tr[{tr}]/td[3]').text).text
#     industry_dict['net_change']=BeautifulSoup(driver.find_element(
#             by=By.XPATH, value=f'//*[@id="ux-page"]/div/div[2]/strong/table/tbody/tr[{tr}]/td[4]').text).text
#     industry_dict['pct_change']=BeautifulSoup(driver.find_element(
#             by=By.XPATH, value=f'//*[@id="ux-page"]/div/div[2]/strong/table/tbody/tr[{tr}]/td[5]').text).text
    
#     industry_list.append(industry_dict)
# ind_df=pd.DataFrame(industry_list)

/var/folders/09/0pcf7rgs7c38nk2clfp7p_5w0000gq/T/ipykernel_20838/1138229553.py:2: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  total_elems=len(driver.find_elements_by_tag_name('tr'))


In [12]:
# ind_df=ind_df.replace(",","",regex=True).replace("%","",regex=True)

In [13]:
# ind_df['month']=ind_df.iloc[0][1].split()[0]

In [14]:
# ind_df=ind_df[1:].reset_index(drop=True)

In [15]:
# ind_df['current_value']=ind_df['current_value'].astype(float)*1000
# ind_df['comp_value']=ind_df['comp_value'].astype(float)*1000
# ind_df['net_change']=ind_df['net_change'].astype(float)*1000
# ind_df['pct_change']=ind_df['pct_change'].astype(float)

In [16]:
# ind_df.month.iloc[0]

'FEB'

In [17]:
# ind_df.to_csv(f'data/{ind_df.month.iloc[0]}_industry_emp_master.csv', index=False)
# ind_df.to_json(f'data/{ind_df.month.iloc[0]}_industry_emp_master.json', orient='records')

# Unemployment rates

In [7]:
rate_url='https://www.bls.gov/regions/new-york-new-jersey/data/xg-tables/ro2xglausnyc.htm'
response=requests.get(rate_url)
doc=BeautifulSoup(response.text, 'html.parser')
table=doc.find("table", {"id": "ro2xglausnyc4"})

ele_list=[]
for ele in table.find('tbody').find_all('tr'):
    for ele1 in ele.find_all('td'):
        ele_dict={}
        ele_dict['year']=ele.find('th').text.strip()
        ele_dict['nyc_rate']=ele1.text.strip()
        ele_list.append(ele_dict)
rate=pd.DataFrame(ele_list)

rate=rate[(rate.nyc_rate !="")].reset_index(drop=True)
rate['nyc_rate']=rate.nyc_rate.str.replace(
    "(r)","").str.replace("(p)","").str.replace("(","").str.replace(")","")

rate['date']=pd.Series(pd.period_range("1/1/2012", freq="M", periods=len(rate))).astype(str)
rate['nyc_rate']=rate['nyc_rate'].astype(float)

rate=rate[['date', 'nyc_rate']]

national_unemp_url = "https://data.bls.gov/timeseries/LNS14000000"

response=requests.get(national_unemp_url)
doc=BeautifulSoup(response.text, 'html.parser')
table=doc.find_all('table', {"id":"table0"})


ele_list=[]

for ele in table:
    for ele1 in ele.find_all('tr'):  
        for ele2 in ele1.find_all('td'):
            ele_dict={}
            ele_dict['year']=ele1.find('th').text.strip()
            ele_dict['us_rate']=ele2.text.strip()
            ele_list.append(ele_dict)            
us_df=pd.DataFrame(ele_list)

us_df=us_df[~(us_df.us_rate =="")]
us_df['us_rate']=us_df['us_rate'].astype(float)
us_df['date']=pd.Series(pd.period_range("1/1/2012", freq="M", periods=len(us_df))).astype(str)
us_df=us_df[['date', 'us_rate']]

merged_jobless=pd.merge(rate,us_df)

merged_jobless.to_json("data/unemployment_rate.json", orient='records')
merged_jobless.to_csv("data/unemployment_rate.csv", index=False)

/var/folders/09/0pcf7rgs7c38nk2clfp7p_5w0000gq/T/ipykernel_4847/1072654595.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  rate['nyc_rate']=rate.nyc_rate.str.replace(
/var/folders/09/0pcf7rgs7c38nk2clfp7p_5w0000gq/T/ipykernel_4847/1072654595.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  rate['nyc_rate']=rate.nyc_rate.str.replace(


In [15]:
# ind_df.sort_values('pct_change', ascending=False).head(30)
# https://dol.ny.gov/labor-statistics-new-york-city-region
# https://dol.ny.gov/system/files/documents/2022/03/2021-significant-industries-new-york-city_0.pdf
# https://data.bls.gov/timeseries/LNS14000000

# Some cleaning

In [86]:
significant_ind = [
   'Mining, Logging and Construction','Couriers and messengers','Transportation and Warehousing',
    'Information', 'Financial Activities','Professional, Scientific, and Technical Services',
    'Administrative and Support Services','Educational Services','Ambulatory Health Care Services',
    'Social Assistance','Food Services and Drinking Places'
]

In [87]:
# https://dol.ny.gov/system/files/documents/2022/03/2021-significant-industries-new-york-city_0.pdf

In [88]:
# download excel file from the link in url
url="https://dol.ny.gov/statistics-new-york-city-employment-statistics"
r= requests.get(url, allow_redirects=True)
open('data/raw_industry_data.xlsx', 'wb').write(r.content)
# read the file
df=pd.read_excel('data/raw_industry_data.xlsx', skiprows=1)

In [89]:
df=df[['AREA', 'YEAR', 'SERIESCODE', 'AREANAME', 'INDUSTRY_TITLE', 'JAN','FEB', 'MAR', 'APR', 'MAY', 'JUN', 
    'JUL', 'AUG', 'SEP', 'OCT', 'NOV','DEC']]

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3929 entries, 0 to 3928
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AREA            3929 non-null   int64  
 1   YEAR            3929 non-null   int64  
 2   SERIESCODE      3929 non-null   int64  
 3   AREANAME        3929 non-null   object 
 4   INDUSTRY_TITLE  3929 non-null   object 
 5   JAN             3929 non-null   float64
 6   FEB             3929 non-null   float64
 7   MAR             3805 non-null   float64
 8   APR             3805 non-null   float64
 9   MAY             3805 non-null   float64
 10  JUN             3805 non-null   float64
 11  JUL             3805 non-null   float64
 12  AUG             3805 non-null   float64
 13  SEP             3805 non-null   float64
 14  OCT             3805 non-null   float64
 15  NOV             3805 non-null   float64
 16  DEC             3805 non-null   float64
dtypes: float64(12), int64(3), object(

In [91]:
df2020=df[(df.YEAR == 2022)]

In [92]:
df2020['mom_change']=((df2020['FEB']- df2020['JAN'])*100/df2020['JAN']).round(1)

/var/folders/09/0pcf7rgs7c38nk2clfp7p_5w0000gq/T/ipykernel_20838/3643508164.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2020['mom_change']=((df2020['FEB']- df2020['JAN'])*100/df2020['JAN']).round(1)


In [93]:
final_df=df2020[df2020.INDUSTRY_TITLE.isin(significant_ind)].reset_index(drop=True)

In [97]:
final_df[["INDUSTRY_TITLE","FEB", 'mom_change']].to_json("data/sig_ind.json", orient='records')

In [95]:
final_df.FEB.sum()/4085.9

0.6654347879292201

In [96]:
final_df[['']]

,AREA,YEAR,SERIESCODE,AREANAME,INDUSTRY_TITLE,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,mom_change
0,93561,2022,15000000,New York City,"Mining, Logging and Construction",130.4,132.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5
1,93561,2022,43400089,New York City,Transportation and Warehousing,130.2,128.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.5
2,93561,2022,50000000,New York City,Information,230.5,232.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,93561,2022,55000000,New York City,Financial Activities,458.8,463.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1
4,93561,2022,60540000,New York City,"Professional, Scientific, and Technical Services",435.4,442.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.6
5,93561,2022,60561000,New York City,Administrative and Support Services,237.7,240.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.3
6,93561,2022,65610000,New York City,Educational Services,236.7,246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.9
7,93561,2022,65621000,New York City,Ambulatory Health Care Services,358.8,365.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
8,93561,2022,65624000,New York City,Social Assistance,214.4,216.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1
9,93561,2022,70722000,New York City,Food Services and Drinking Places,248.0,249.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8
